In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional     scklearn functions
from sklearn.model_selection import GridSearchCV, train_test_split   #Perforing grid search

import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

file = os.path.join('data', 'processed_data.csv')

train = pd.read_csv(file)
target = 'y'
IDcol = 'ID'
train

,ID,age,job0,job1,job2,job3,job4,job5,job6,job7,...,month,duration,campaign,pdays,previous,poutcome0,poutcome1,poutcome2,poutcome3,y
0,1,0.324675,0,0,0,0,1,0,0,0,...,0.363636,0.038650,0.018519,0.000000,0.000000,0,0,0,1,0
1,2,0.311688,0,0,0,0,0,0,0,0,...,0.272727,0.025509,0.000000,0.294737,0.007273,0,1,0,0,0
2,3,0.376623,1,0,0,0,0,0,0,0,...,0.545455,0.019840,0.018519,0.000000,0.000000,0,0,0,1,0
3,4,0.129870,0,0,0,0,1,0,0,0,...,0.545455,0.044834,0.018519,0.000000,0.000000,0,0,0,1,0
4,5,0.311688,0,0,0,0,0,0,0,0,...,0.363636,0.048183,0.074074,0.000000,0.000000,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25312,25313,0.480519,0,1,0,0,0,0,0,0,...,0.363636,0.220046,0.018519,0.422222,0.003636,1,0,0,0,1
25313,25314,0.441558,0,0,0,0,0,0,0,1,...,0.090909,0.057202,0.000000,0.647953,0.014545,1,0,0,0,1
25314,25315,0.220779,0,1,0,0,0,0,0,0,...,0.363636,0.243494,0.018519,0.000000,0.000000,0,0,0,1,1
25315,25316,0.246753,0,0,1,0,0,0,0,0,...,0.909091,0.299923,0.018519,0.000000,0.000000,0,0,0,1,1


In [9]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
        print('n_estimators=' + str(cvresult.shape[0]))

    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['y'],eval_metric='auc')

    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]

    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['y'].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['y'], dtrain_predprob))

#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')

predictors = [x for x in train.columns if x not in [target, IDcol]]
print(predictors)
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=15,
 seed=27)
modelfit(xgb1, train, predictors)

['age', 'job0', 'job1', 'job2', 'job3', 'job4', 'job5', 'job6', 'job7', 'job8', 'job9', 'job10', 'job11', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome0', 'poutcome1', 'poutcome2', 'poutcome3']
n_estimators=154

Model Report
Accuracy : 0.8445
AUC Score (Train): 0.965297


In [18]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(3,10,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=154, max_depth=5, min_child_weight=1,
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=15, seed=27),
                        param_grid = param_test1, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.cv_results_['std_test_score'], gsearch1.best_params_, gsearch1.best_score_

(array([0.00284818, 0.00260134, 0.00263753, 0.00272653, 0.0041585 ,
        0.00308595, 0.0028884 , 0.00294104, 0.00474208, 0.00390615,
        0.00338937, 0.00315047, 0.00324363, 0.00328381, 0.00325775,
        0.00342659]),
 {'max_depth': 7, 'min_child_weight': 9},
 0.9330744542639045)

In [19]:
param_test1 = {
 'max_depth':[6, 7, 8],
 'min_child_weight':[8, 9, 10]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=154, max_depth=7, min_child_weight=9,
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=15, seed=27),
                        param_grid = param_test1, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.cv_results_['std_test_score'], gsearch1.best_params_, gsearch1.best_score_

(array([0.00340382, 0.00332451, 0.00286534, 0.00369496, 0.00315047,
        0.00337153, 0.00284067, 0.00380209, 0.00374967]),
 {'max_depth': 6, 'min_child_weight': 10},
 0.933372818195221)

In [20]:
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=154, max_depth=6, min_child_weight=10,
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=15, seed=27),
                        param_grid = param_test3, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.cv_results_['std_test_score'], gsearch1.best_params_, gsearch1.best_score_

(array([0.00286534, 0.00280689, 0.00283046, 0.0034603 , 0.00340097]),
 {'gamma': 0.4},
 0.9334458445413716)

In [22]:
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=154, max_depth=6, min_child_weight=10,
                                                  gamma=0.4, subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=15, seed=27),
                        param_grid = param_test4, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.cv_results_['std_test_score'], gsearch1.best_params_, gsearch1.best_score_

(array([0.00358652, 0.00381167, 0.00433528, 0.00293296, 0.00352425,
        0.00357326, 0.00315449, 0.00299664, 0.00366866, 0.00281109,
        0.00340097, 0.00309309, 0.00334159, 0.00312407, 0.0030473 ,
        0.00294517]),
 {'colsample_bytree': 0.7, 'subsample': 0.9},
 0.9335160544201864)

In [23]:
param_test5 = {
 'subsample':[i/100.0 for i in range(85,105,5)],
 'colsample_bytree':[i/100.0 for i in range(85,105,5)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=154, max_depth=6, min_child_weight=10,
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=15, seed=27),
                        param_grid = param_test5, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.cv_results_['std_test_score'], gsearch1.best_params_, gsearch1.best_score_

(array([0.00326728, 0.00349676, 0.00342134, 0.00327866, 0.0029175 ,
        0.00302577, 0.00328133, 0.00445897, 0.00312227, 0.00351334,
        0.00335024, 0.00397438, 0.00281895, 0.00290423, 0.00309219,
        0.00356375]),
 {'colsample_bytree': 0.85, 'subsample': 0.85},
 0.934008646218549)

In [24]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=154, max_depth=6, min_child_weight=10,
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=15, seed=27),
                        param_grid = param_test6, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.cv_results_['std_test_score'], gsearch1.best_params_, gsearch1.best_score_

(array([0.00286534, 0.0037598 , 0.00338186, 0.00302896, 0.00356986]),
 {'reg_alpha': 1e-05},
 0.933372818195221)

In [26]:
param_test7 = {
 'scale_pos_weight':range(6, 20)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=154, max_depth=6, min_child_weight=10,
                                                  gamma=0.4, subsample=0.85, colsample_bytree=0.85,objective= 'binary:logistic', 
                                                  nthread=4, scale_pos_weight=15, seed=27),
                        param_grid = param_test7, scoring='roc_auc', n_jobs=4, iid=False, cv=5)
gsearch1.fit(train[predictors],train[target])
gsearch1.cv_results_['std_test_score'], gsearch1.best_params_, gsearch1.best_score_

(array([0.00307478, 0.00314315, 0.00354703, 0.00334295, 0.00281726,
        0.00329312, 0.00357834, 0.0033461 , 0.00357459, 0.002816  ,
        0.00393624, 0.00327671, 0.0030966 , 0.00350898]),
 {'scale_pos_weight': 8},
 0.9344479173078591)